# Deep Learning: Mnist Analysis

In [56]:
%matplotlib inline
import math
import numpy as np
import utils; reload(utils)
from utils import *

from sympy import Symbol
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Lambda, Dense
from matplotlib import pyplot as plt

In [57]:
np.random.seed(1)

In [58]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [59]:
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [60]:
# Keras needs to have at least one channel (color), so we expand the dimensions here.
X_test = np.expand_dims(X_test,1)
X_train = np.expand_dims(X_train,1)

In [61]:
# We would like to have an output in the form: [0, 0, 1, 0...] so we transform the labels with
# "onehot".
y_train = onehot(y_train)
y_test = onehot(y_test)

In [22]:
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)

In [23]:
# We normalize the inputs so the training is more stable.
def norm_input(x): return (x-mean_px)/std_px

In [62]:
#def sigmoid(x):
#  return 1 / (1 + math.exp(-x))
#def sigmoid_prime(x):
#  return math.e/(1 - math.e * x)**2

In [63]:
# We could have used the "sgd" function included in Keras, but for the sake of learning we
# are going to implement our own function.
#def sse(y, y_pred): return ((y-y_pred)**2).sum()
#def loss(y, a, b, X): return sse(y, lin(a, b, X))
#def avg_loss(y, a, b, X): return np.sqrt(loss(y, a, b, X)/n)

#W = 
#def update(lr=0.01):
    
    

In [64]:
model = Sequential([
    Lambda(norm_input, input_shape=(1,28,28)),
    Flatten(),
    Dense(10, activation='softmax')
])
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['accuracy'])

/Users/javidgon/anaconda/lib/python2.7/site-packages/keras/layers/core.py:577: UserWarning: `output_shape` argument not specified for layer lambda_6 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [ ]:
# This creates batches based on images in "array-form". This class is also quite powerful
# as it allows us to do Data Augmentation.
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [ ]:
# We train the batches in the model
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)

In [ ]:
# We increase the learning rate until we get overfitting.
lm.optimizer.lr=0.1

In [ ]:
# We train the batches in the model
model.fit_generator(batches, batches.N, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.N)